Data is pre-processed in Java by splitting it into sentences, each on a new line, and lowering the case of all files. New files are saved as TXT with "\_SENTbySENT" added at the end of the file name

In [1]:
# Import all necesssry modules
import os, sys, os.path
from stat import *
import errno
import glob
import csv
import re
import shutil
import csv

## Count collected data files 

In [2]:
#path = r"C:\Users\ani\Desktop\Course data Thesis\PROCESSED\all coures to select from"   # home laptop
path = r"C:\Users\a.dimitrova\Desktop\Course data Thesis\PROCESSED\Selected files TRAIN"   # toshiba laptop
#import os
import os.path

allFiles = sum([len(files) for r, d, files in os.walk(path)])

print("All files: ", allFiles)

# ---------------
def fileCounter(folder):    
    count = 0
    
    for filename in os.listdir(folder):
        path = os.path.join(folder, filename)
        
        if os.path.isfile(path):
            if os.path.isfile(path):
                name = os.path.basename(path)
                if name.endswith('.txt'):
                    count += 1
        elif os.path.isdir(path):
            count += fileCounter(path)
            
    return count
# ---------------

"""
TXT files:  563
SRT files:  568
RTF files:  562
All Files:  1693
"""

print("TXT files: ", fileCounter(root))

All files:  113


NameError: name 'root' is not defined

## List files in directories

In [ ]:
#files = glob.glob(root)

def traverseDir(rootIn):
# traverse root directory, and list directories as dirs and files as files
    for rootIn, dirs, files in os.walk("."):
        path = rootIn.split(os.sep)
        print((len(path) - 1) * '---', os.path.basename(root))
        for file in files:
            print(len(path) * '---', file)
            
traverseDir(root)

## Count all sentences in all files

### Sentence and labels count + printing labeled sentences

## --STATS--

In [53]:
#path = r"C:\Users\ani\Desktop\Course data Thesis\PROCESSED\all coures to select from"   #THE REST OF THE DATA
#path = r"C:\Users\ani\Desktop\Course data Thesis\PROCESSED\Selected files TRAIN"    #TRAIN / TEST
#path = r"C:\Users\ani\Desktop\Course data Thesis\PROCESSED\Selected files TRAIN"   # toshiba laptop
path = r"C:\Users\ani\Desktop\Course data Thesis\Intent Mining ALL files"

fileExtention = "_labels.txt"

"""
TRAIN:     <<< FILES: 111 | SENTENCES: 8835 | FOLDERS: 10 | TUPLES: 8724 >>>
THE REST:  <<< FILES: 556 | SENTENCES: 38482 | FOLDERS: 320 | TUPLES: 9 >>>
"""

sentCounter = 0
finalSentCounter = 0  # to be used for sentences WITHOUT counting the NL ones
fileCounter = 0
folderCounter = 0
allTuples = []   # holds tuples with LABEL | SENTENCE for all valid labelled sentences while the rest is removed, so this is
# more accurate metric for totalsentence number AND IGNORES THE NL sentences
unlabeledSent = 0
unlabeledFiles = 0

curFilePath = ""

for root, subdirs, files in os.walk(path):

    for curFile in os.listdir(root):

        curFilePath = os.path.join(root, curFile)

        if os.path.isdir(curFilePath):
            folderCounter += 1
            pass

        elif curFilePath.endswith(fileExtention):
            try:
                fileCounter += 1
                curFile = open(curFilePath, 'r', encoding = "ISO-8859-1") #IMPORTANT ENCODING! UTF8 DOESN'T WORK

                fcontent = curFile.read().split("\n")
                
                for sent in fcontent:
                    if sent == "[music]." or sent == "[music]":
                        pass
                    else:
                        sentParts = sent.split("|")
                        if len(sentParts) == 1:
                            #PASS if sentence has no label or is an empty line
                            pass
                        else: 
                            sentTuple = (sentParts[0],sentParts[1])
                            if sentParts[0] != "NL":
                                allTuples.append(sentTuple)
                            if sentTuple[0] == '':
                                #Add to unlabeledSent if sentence has no label, but is split in 2
                                unlabeledSent += 1
                            
                        #print(sentParts[0],"<<>>",sentParts[1])
                        sentCounter += 1
                        
            except FileNotFoundError as e:
                print("SKIPPING [{}]: {}".format(e.strerror,curFile))
                pass
            
        else:
            pass

        
finalSentCounter = sentCounter - NLtuples   #without considering the NL tuples
        
       
# count sentences labeled with specific label
CDtuples = 0
CMtuples = 0
SMtuples = 0
APtuples = 0
EXtuples = 0
NLtuples = 0
wrongTuples = 0

labels = ['CD','CM','SM','AP','EX','NL']

for tup in allTuples:
    if "CD" in tup: CDtuples += 1
    if "CM" in tup: CMtuples += 1
    if "SM" in tup: SMtuples += 1
    if "AP" in tup: APtuples += 1
    if "EX" in tup: EXtuples += 1
    if "NL" in tup: NLtuples += 1     
        
percentCD = (CDtuples / len(allTuples)) * 100
percentCM = (CMtuples / len(allTuples)) * 100
percentAP = (APtuples / len(allTuples)) * 100
percentEX = (EXtuples / len(allTuples)) * 100
percentSM = (SMtuples / len(allTuples)) * 100
#percentNL = (NLtuples / len(allTuples-NLtuples)) * 100

print("____________________________________________")
print("|CD     |CM    |SM    |AP    |EX    |NL    | \n|{}   |{}   |{}   |{}   |{}   |{}   | ".format(CDtuples,CMtuples,SMtuples,APtuples,EXtuples,NLtuples))
print("|{0:.2f}% |{1:.2f}% |{2:.2f}% |{3:.2f}% |{4:.2f}% |{5:.2f}% |".format(percentCD,percentCM,percentSM,percentAP,percentEX,percentNL))
print("¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯")

print("_____________________________________________")
print("| FILES: {} |\n| SENTENCES_АLL: {} | SENTENCES_REL: {} |".format(fileCounter, finalSentCounter, len(allTuples)))
print("| LABELED SENT: {} | UNLABELED SENT: {} |".format((finalSentCounter - unlabeledSent),unlabeledSent))
print("¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯")

print("____________________________________")
print("|CD     |CM    |SM    |AP    |EX    | \n|{}   |{}   |{}   |{}   |{}   | ".format(CDtuples,CMtuples,SMtuples,APtuples,EXtuples))
print("|{0:.2f}% |{1:.2f}% |{2:.2f}% |{3:.2f}% |{4:.2f}% |".format(percentCD,percentCM,percentSM,percentAP,percentEX))
print("¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯")



################## DISPLAY DATA FOR SPECIFIC LABEL #############
tupType = "cm".upper()    #convert to Upper case otherwise it shows no res
num = 0
displayNum = 5

print("___________________________________________________________________________________________")
print("\n-------PRINTING the first {} <<{}>> TUPLES------\n".format(displayNum,tupType))
for tup in allTuples:
    if tupType in tup: 
        if num < displayNum: 
            print(tup)
            num += 1

____________________________________________
|CD     |CM    |SM    |AP    |EX    |NL    | 
|6621   |304   |570   |318   |566   |0   | 
|78.99% |3.63% |6.80% |3.79% |6.75% |0.00% |
¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯
_____________________________________________
| FILES: 111 |
| SENTENCES_АLL: 8960 | SENTENCES_REL: 8382 |
| LABELED SENT: 8958 | UNLABELED SENT: 2 |
¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯
____________________________________
|CD     |CM    |SM    |AP    |EX    | 
|6621   |304   |570   |318   |566   | 
|78.99% |3.63% |6.80% |3.79% |6.75% |
¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯
___________________________________________________________________________________________

-------PRINTING the first 5 <<CM>> TUPLES------

('CM', 'this is the average speed of the ball ')
('CM', "speed is how far you've traveled over how long it took you so if i divide these this is the speed of the ball the average speed of the ball between point four and point five seconds and this

### Select every 5th file for labeling

#### Collecting files for TRAIN dataset which, on its end will be split around [70% : 30%] for Train and Test data once labeled

In [ ]:
path = r"C:\Users\ani\Desktop\Course data Thesis\PROCESSED\all coures to select from"

fileCounter = 0

for root, subdirs, files in os.walk(path):

    for curFile in os.listdir(root):

        curFilePath = os.path.join(root, curFile)

        if os.path.isdir(curFilePath):
            pass

        else:
            fileCounter += 1
            # get every 5th file among the 556 files, so that I have approx. equal ratio based on the files in each course
            if fileCounter % 5 == 0 or fileCounter == 1:  
                newDestinationRoot = r"C:\Users\ani\Desktop\Course data Thesis\PROCESSED\Selected files TRAIN"
                curFile = open(curFilePath, 'r', encoding = "ISO-8859-1") #IMPORTANT ENCODING! UTF8 DOESN'T WORK

                #print("File {} | {} ".format(fileCounter, curFile.name))
                
                # split the original path to parts and take the course folder name
                # combine it with the file name and put all files in a single course folder
                fileParts = curFile.name.split("\\")
                newLocation = newDestinationRoot+'\\'+fileParts[7]+'\\'+fileParts[-1]
                
                # get the path of the new directory to check if it exists
                directory = newDestinationRoot+"\\"+fileParts[7]
                print(directory)
                
                # if the dir doesn't exist, create it
                if not os.path.exists(directory):
                    os.makedirs(directory)
                    
                # close file otherwise throws an err
                curFile.close()                
                
                # copy the files
                #print("COPYING FILE #{}.. \nSOURCE:\t\t {}\nDESTINATION:\t {} \n".format(fileCounter,curFile.name,newLocation))
                ##### UNCOMMENT TO COPY FILES ##### 
                #shutil.copy(curFile.name, newLocation)
                

#print("\n<<< FILES: {} | SENTENCES: {} | FOLDERS: {} >>>".format(fileCounter, sentCounter, folderCounter))

### Check for empty files

In [ ]:
path = r"C:\Users\ani\Desktop\Course data Thesis\PROCESSED\Data for Crowdsourcing - UNLABELED"

fileCounter = 0

for root, subdirs, files in os.walk(path):

    for curFile in os.listdir(root):

        curFilePath = os.path.join(root, curFile)

        if os.path.isdir(curFilePath):
            pass

        else:
            
            curFile = open(curFilePath, 'r', encoding = "ISO-8859-1") #IMPORTANT ENCODING! UTF8 DOESN'T WORK

            lineCount = 0
            
            for line in curFile.read().split("\n"):
                lineCount += 1
                
            if lineCount < 2:
                print("Empty file: ", os.path.basename(curFilePath))
            
            curFile.close()                

### Splitting into HITS of 5 files each

In [ ]:
path = r"C:\Users\ani\Desktop\Course data Thesis\PROCESSED\all coures to select from"

fileCounter = 0
foldIndex = 0

#folders = list(filter(lambda x: x%5 == 0, range[0:112])
folderNum = range(1, range[0:112]+1)

for root, subdirs, files in os.walk(path):

    for curFile in os.listdir(root):

        curFilePath = os.path.join(root, curFile)

        if os.path.isdir(curFilePath):
            pass

        else:
            fileCounter += 1
            newDestinationRoot = r"C:\Users\ani\Desktop\Course data Thesis\PROCESSED\Data\Hit"
            
            if not fileCounter % 5 == 0:  
                print(newDestinationRoot)
            elif fileCounter % 5 == 0: 
                newDestinationRoot = newDestinationRoot + folderNum(foldIndex)
                if not folderNum == len(folderNum)+1:
                    foldIndex += 1
                    print(newDestinationRoot)

            curFile = open(curFilePath, 'r', encoding = "ISO-8859-1") #IMPORTANT ENCODING! UTF8 DOESN'T WORK
                
            # split the original path to parts and take the course folder name
            # combine it with the file name and put all files in a single course folder
            newLocation = newDestinationRoot
            print(newLocation)
            
            # if the dir doesn't exist, create it
            if not os.path.exists(newDestinationRoot):
                os.makedirs(newDestinationRoot)

            # close file otherwise throws an err
            curFile.close()                

            # copy the files
            #print("COPYING FILE #{}.. \nSOURCE:\t\t {}\nDESTINATION:\t {} \n".format(fileCounter,curFile.name,newLocation))
            ##### UNCOMMENT TO COPY FILES ##### 
            #shutil.copy(curFile.name, newLocation)
                

#print("\n<<< FILES: {} | SENTENCES: {} | FOLDERS: {} >>>".format(fileCounter, sentCounter, folderCounter))

## Convert all files to CSV

In [8]:
path = r"C:\Users\ani\Desktop\Course data Thesis\PROCESSED\Data for Crowdsourcing - UNLABELED"

data = []

### go over the files and convert them to CSV   

for root, subdirs, files in os.walk(path):

    for curFile in os.listdir(root):

        curFilePath = os.path.join(root, curFile)

        if os.path.isdir(curFilePath):
            pass

        else:
            if curFilePath.endswith(".txt"):
                curFile = open(curFilePath, 'r', encoding = "ISO-8859-1") #IMPORTANT ENCODING! UTF8 DOESN'T WORK
                fileExtRemoved = os.path.splitext(os.path.abspath(curFilePath))[0]
                #print(fileExtRemoved)
                fcontent = curFile.read().split("\n")

                with open(fileExtRemoved+'.csv', 'w') as OutpCSV:
                    converter = csv.writer(OutpCSV, delimiter=',')
                    #converter.writerow(["---","-----"])
                    #converter.writerow(["LABEL","SENTENCE"])
                    for line in fcontent:
                        line = line.split("|")
                        if len(line) > 1:
                            converter.writerow([" ",line[1]])
                        else:
                            pass
            else:
                pass

In [9]:
path = r"C:\Users\ani\Desktop\Course data Thesis\PROCESSED\Data for Crowdsourcing - UNLABELED"
### concatenate files into one         
                    
from glob import glob

with open(path+'\singleDataFile2.csv','a',encoding = "utf-8") as singleFile:
    for root, subdirs, files in os.walk(path):
        for curFile in os.listdir(root):
            curFilePath = os.path.join(root, curFile)
            if os.path.isdir(curFilePath):
                pass
            else:
                if curFilePath.endswith(".csv"):
                    with open(curFilePath, 'r', encoding = "ISO-8859-1") as iFile:
                        next(iFile)
                        for line in iFile:
                            if len(line) == 1:
                                pass
                            else:
                                #print(line)
                                singleFile.write(line)
                else:
                    pass

KeyboardInterrupt: 

In [4]:
path = r"C:\Users\ani\Desktop\Course data Thesis\Intent Mining ALL files"

data = []

### go over the files and convert them to CSV   

text = ""

with open(r"C:\Users\ani\Desktop\Course data Thesis\INTENT MINING\allMergedLabeled.txt", "w", encoding = "utf-8") as oFile:
    for root, subdirs, files in os.walk(path):

        for curFile in os.listdir(root):

            curFilePath = os.path.join(root, curFile)

            if os.path.isdir(curFilePath):
                pass

            else:
                if curFilePath.endswith(".txt"):
                    if not curFilePath.endswith("_AutoRuleLabels.txt"):
                        continue
                    else:
                        iFile = open(curFilePath, 'r', encoding = "ISO-8859-1")
                        sentList = iFile.read().split('\n')

                        for sent in sentList:
                            if len(sent) > 0:
                                oFile.write(sent+"\n")
                            else:
                                continue
                else:
                    continue

#with open(r'C:\Users\ani\Desktop\Course data Thesis\Intent Mining ALL files\allMerged.txt', 'w') as oFile:
    
textsent = text.split("\n")